# Accessing the WiGLE API with Python

This notebook demonstrates how to access the [WiGLE API](https://api.wigle.net/) for crowd-sourced information on the location of WiFi access points.

https://medium.com/@neuralnets/building-a-wifi-spots-map-of-networks-around-you-with-wigle-and-python-5adf72a48140

#### Import Packages

In [1]:
import requests
from requests.auth import HTTPBasicAuth
import pandas as pd
from pandas.io.json import json_normalize

In [2]:
#Wigle Authentication
api_name = 'AID18c60572f3b545f8be5573d53e3e0609'
api_token = '0ae1905e583e68160d759c2da6f02c4b'

#### Request coordinates for area of interest

In [3]:
#Search parameters
payload = {
    'addresscode': 'Dublin',
    'api_name': (api_name + api_token)
}

#Call the WiGLE geocode endpoint
coords = requests.get(url='https://api.wigle.net/api/v2/network/geocode', params=payload, auth=HTTPBasicAuth(api_name, api_token)).json()
#Print the response
print(coords)

{'success': True, 'results': [{'address': {'city': 'County Dublin', 'county': 'County Dublin', 'state_district': 'Leinster', 'country': 'Ireland', 'country_code': 'ie'}, 'lat': 53.3497645, 'lon': -6.2602732, 'importance': 0.83287508674498, 'place_id': 174879444, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://www.openstreetmap.org/copyright', 'osm_type': 'relation', 'display_name': 'Dublin, County Dublin, Leinster, Ireland', 'boundingbox': [53.2987342, 53.4105416, -6.3870259, -6.1148829]}]}


#### Requst data using coordiantes

In [15]:
payload = {
    'onlymine': 'false',
    'freenet': 'false',
    'paynet': 'false',
    'latrange1':'53.2987342', 
    'latrange2':'53.4105416',
    'longrange1':'-6.3870259',
    'longrange2':'-6.1148829',
    'resultsPerPage':'100',
    'searchAfter':'0',
    'api_key': (api_name + api_token)
}

In [16]:
#Call the WiGLE search endpoint
wifi_locations = requests.get(url='https://api.wigle.net/api/v2/network/search', params=payload, auth=HTTPBasicAuth(api_name, api_token)).json()
print(wifi_locations)

{'success': True, 'totalResults': 460135, 'first': 1, 'last': 100, 'resultCount': 100, 'results': [{'trilat': 53.33297348, 'trilong': -6.26166296, 'ssid': 'Pichichu Network', 'qos': 0, 'transid': '20161024-00000', 'firsttime': '2016-10-09T20:00:00.000Z', 'lasttime': '2016-10-24T09:00:00.000Z', 'lastupdt': '2016-10-24T09:00:00.000Z', 'netid': '00:00:00:D8:E6:1B', 'name': None, 'type': 'infra', 'comment': None, 'wep': '?', 'bcninterval': 0, 'freenet': '?', 'dhcp': '?', 'paynet': '?', 'userfound': False, 'channel': 1, 'encryption': 'unknown', 'country': 'IE', 'region': None, 'housenumber': None, 'road': 'Adelaide Road', 'city': 'Dublin', 'postalcode': '2'}, {'trilat': 53.33297729, 'trilong': -6.29648495, 'ssid': '5424 3661', 'qos': 0, 'transid': '20080202-00000', 'firsttime': '2008-02-02T10:00:00.000Z', 'lasttime': '2008-02-02T07:00:00.000Z', 'lastupdt': '2008-02-02T05:00:00.000Z', 'netid': '00:00:C5:B1:47:75', 'name': None, 'type': 'BSS', 'comment': None, 'wep': 'Y', 'bcninterval': 0, 'f

#### Requst data using city name and country

In [62]:
payload = {
    'onlymine': 'false',
    'freenet': 'false',
    'paynet': 'false',
    'city': 'Dublin',
    'country': 'IE',
    'resultsPerPage':'100',
    'searchAfter':'0',
    'api_key': (api_name + api_token)
}

In [53]:
#Call the WiGLE search endpoint
wifi_locations = requests.get(url='https://api.wigle.net/api/v2/network/search', params=payload, auth=HTTPBasicAuth(api_name, api_token)).json()
print(wifi_locations)

{'success': True, 'totalResults': 0, 'first': 1, 'last': 0, 'resultCount': 0, 'results': [], 'searchAfter': None, 'search_after': None}


#### Place results in a data frame

In [33]:
#Place data in the dataframe
df_wifi = json_normalize(wifi_locations['results'])
#Rename Columns
df_wifi = df_wifi.rename(columns={'trilat': 'lat', 'trilong': 'lon'})

In [38]:
df_wifi.head()

,bcninterval,channel,city,comment,country,dhcp,encryption,firsttime,freenet,housenumber,...,qos,region,road,ssid,transid,lat,lon,type,userfound,wep
0,0,1,Dublin,None,IE,?,unknown,2016-10-09T20:00:00.000Z,?,None,...,0,None,Adelaide Road,Pichichu Network,20161024-00000,53.332973,-6.261663,infra,False,?
1,0,6,Dublin,None,IE,?,wep,2008-02-02T10:00:00.000Z,?,None,...,0,None,Dolphin Road,5424 3661,20080202-00000,53.332977,-6.296485,BSS,False,Y
2,0,6,Dublin,None,IE,?,unknown,2010-10-28T18:00:00.000Z,?,None,...,0,None,Priorswood Road,5424 4155,20101028-00000,53.399746,-6.197673,None,False,?
3,0,7,Dublin,None,IE,?,none,2001-01-01T00:00:00.000Z,?,22b,...,4,None,Sandford Road,5643 6521,20070405-00000,53.322994,-6.249116,BSS,False,N
4,0,7,Dublin,None,IE,?,none,2001-01-01T00:00:00.000Z,?,None,...,0,None,Patrick Street,5662 2771,20070312-00000,53.340137,-6.271980,None,False,N


#### API Request function

In [3]:
#Test Payload
payload = {
    'onlymine': 'false',
    'freenet': 'false',
    'paynet': 'false',
    'road':'Rathmines Road',
    #'latrange1':'53.2987342', 
    #'latrange2':'53.4105416',
    #'longrange1':'-6.3870259',
    #'longrange2':'-6.1148829',
    #'resultsPerPage':'100',
    'searchAfter':'0',
    'api_key': (api_name + api_token)
}

In [ ]:
url='https://api.wigle.net/api/v2/network/search'

has_last_record = False

#Call the WiGLE search endpoint iteratively
def get_wigle_data(url):
    r = requests.get(url, params=payload, auth=HTTPBasicAuth(api_name, api_token)).json()
    print(r)
    last_record = r["searchAfter"]
    if last_record != 'None':
        has_last_record = True
        payload["searchAfter"] = last_record
        print('Last Record is = ' + last_record)
        while has_last_record:
            r = requests.get(url, params=payload, auth=HTTPBasicAuth(api_name, api_token)).json()
            print(r)
            if last_record != 'None':
                if last_record != 'Null':
                    has_last_record = False
                    print('There are no more records')
                else:
                    has_last_record = True
                    payload["searchAfter"] = last_record
                    print('Last Record is = ' + last_record)
    else:
        has_last_record = False
        print('Warning! There are no records to print for the selected search parameters:' + payload)
        
        
get_wigle_data(url)
print(call_count)

{'success': True, 'totalResults': 611, 'first': 1, 'last': 100, 'resultCount': 100, 'results': [{'trilat': -37.8198967, 'trilong': 145.04560852, 'ssid': 'OPTUS_E3CE6F', 'qos': 7, 'transid': '20140929-00000', 'firsttime': '2014-09-29T14:00:00.000Z', 'lasttime': '2018-10-03T15:00:00.000Z', 'lastupdt': '2018-10-03T15:00:00.000Z', 'netid': 'C8:91:F9:E3:CE:70', 'name': None, 'type': 'infra', 'comment': None, 'wep': '2', 'bcninterval': 0, 'freenet': '?', 'dhcp': '?', 'paynet': '?', 'userfound': False, 'channel': 1, 'encryption': 'wpa2', 'country': 'AU', 'region': 'Victoria', 'housenumber': None, 'road': 'Rathmines Road', 'city': None, 'postalcode': '3122'}, {'trilat': -37.81996918, 'trilong': 145.04570007, 'ssid': 'WLAN_B500', 'qos': 2, 'transid': '20180516-00000', 'firsttime': '2018-05-16T19:00:00.000Z', 'lasttime': '2018-10-03T15:00:00.000Z', 'lastupdt': '2018-10-03T15:00:00.000Z', 'netid': 'C8:D0:19:01:B5:00', 'name': None, 'type': 'infra', 'comment': None, 'wep': '2', 'bcninterval': 0, '

{'success': True, 'totalResults': 611, 'first': 1, 'last': 11, 'resultCount': 11, 'results': [{'trilat': -37.82051849, 'trilong': 145.05123901, 'ssid': None, 'qos': 0, 'transid': '20181003-00000', 'firsttime': '2018-10-03T18:00:00.000Z', 'lasttime': '2018-10-03T15:00:00.000Z', 'lastupdt': '2018-10-03T15:00:00.000Z', 'netid': 'FA:8F:CA:72:8B:42', 'name': None, 'type': 'infra', 'comment': None, 'wep': '?', 'bcninterval': 0, 'freenet': '?', 'dhcp': '?', 'paynet': '?', 'userfound': False, 'channel': 6, 'encryption': 'unknown', 'country': 'AU', 'region': 'Victoria', 'housenumber': '124', 'road': 'Rathmines Road', 'city': None, 'postalcode': '3123'}, {'trilat': -37.82095718, 'trilong': 145.05506897, 'ssid': None, 'qos': 2, 'transid': '20180605-00000', 'firsttime': '2018-06-05T18:00:00.000Z', 'lasttime': '2018-10-03T15:00:00.000Z', 'lastupdt': '2018-10-03T15:00:00.000Z', 'netid': 'FA:8F:CA:89:88:9C', 'name': None, 'type': 'infra', 'comment': None, 'wep': '?', 'bcninterval': 0, 'freenet': '?',

{'success': True, 'totalResults': 611, 'first': 1, 'last': 11, 'resultCount': 11, 'results': [{'trilat': -37.82051849, 'trilong': 145.05123901, 'ssid': None, 'qos': 0, 'transid': '20181003-00000', 'firsttime': '2018-10-03T18:00:00.000Z', 'lasttime': '2018-10-03T15:00:00.000Z', 'lastupdt': '2018-10-03T15:00:00.000Z', 'netid': 'FA:8F:CA:72:8B:42', 'name': None, 'type': 'infra', 'comment': None, 'wep': '?', 'bcninterval': 0, 'freenet': '?', 'dhcp': '?', 'paynet': '?', 'userfound': False, 'channel': 6, 'encryption': 'unknown', 'country': 'AU', 'region': 'Victoria', 'housenumber': '124', 'road': 'Rathmines Road', 'city': None, 'postalcode': '3123'}, {'trilat': -37.82095718, 'trilong': 145.05506897, 'ssid': None, 'qos': 2, 'transid': '20180605-00000', 'firsttime': '2018-06-05T18:00:00.000Z', 'lasttime': '2018-10-03T15:00:00.000Z', 'lastupdt': '2018-10-03T15:00:00.000Z', 'netid': 'FA:8F:CA:89:88:9C', 'name': None, 'type': 'infra', 'comment': None, 'wep': '?', 'bcninterval': 0, 'freenet': '?',

{'success': True, 'totalResults': 611, 'first': 1, 'last': 11, 'resultCount': 11, 'results': [{'trilat': -37.82051849, 'trilong': 145.05123901, 'ssid': None, 'qos': 0, 'transid': '20181003-00000', 'firsttime': '2018-10-03T18:00:00.000Z', 'lasttime': '2018-10-03T15:00:00.000Z', 'lastupdt': '2018-10-03T15:00:00.000Z', 'netid': 'FA:8F:CA:72:8B:42', 'name': None, 'type': 'infra', 'comment': None, 'wep': '?', 'bcninterval': 0, 'freenet': '?', 'dhcp': '?', 'paynet': '?', 'userfound': False, 'channel': 6, 'encryption': 'unknown', 'country': 'AU', 'region': 'Victoria', 'housenumber': '124', 'road': 'Rathmines Road', 'city': None, 'postalcode': '3123'}, {'trilat': -37.82095718, 'trilong': 145.05506897, 'ssid': None, 'qos': 2, 'transid': '20180605-00000', 'firsttime': '2018-06-05T18:00:00.000Z', 'lasttime': '2018-10-03T15:00:00.000Z', 'lastupdt': '2018-10-03T15:00:00.000Z', 'netid': 'FA:8F:CA:89:88:9C', 'name': None, 'type': 'infra', 'comment': None, 'wep': '?', 'bcninterval': 0, 'freenet': '?',

{'success': True, 'totalResults': 611, 'first': 1, 'last': 11, 'resultCount': 11, 'results': [{'trilat': -37.82051849, 'trilong': 145.05123901, 'ssid': None, 'qos': 0, 'transid': '20181003-00000', 'firsttime': '2018-10-03T18:00:00.000Z', 'lasttime': '2018-10-03T15:00:00.000Z', 'lastupdt': '2018-10-03T15:00:00.000Z', 'netid': 'FA:8F:CA:72:8B:42', 'name': None, 'type': 'infra', 'comment': None, 'wep': '?', 'bcninterval': 0, 'freenet': '?', 'dhcp': '?', 'paynet': '?', 'userfound': False, 'channel': 6, 'encryption': 'unknown', 'country': 'AU', 'region': 'Victoria', 'housenumber': '124', 'road': 'Rathmines Road', 'city': None, 'postalcode': '3123'}, {'trilat': -37.82095718, 'trilong': 145.05506897, 'ssid': None, 'qos': 2, 'transid': '20180605-00000', 'firsttime': '2018-06-05T18:00:00.000Z', 'lasttime': '2018-10-03T15:00:00.000Z', 'lastupdt': '2018-10-03T15:00:00.000Z', 'netid': 'FA:8F:CA:89:88:9C', 'name': None, 'type': 'infra', 'comment': None, 'wep': '?', 'bcninterval': 0, 'freenet': '?',

...

{'success': True, 'totalResults': 611, 'first': 1, 'last': 11, 'resultCount': 11, 'results': [{'trilat': -37.82051849, 'trilong': 145.05123901, 'ssid': None, 'qos': 0, 'transid': '20181003-00000', 'firsttime': '2018-10-03T18:00:00.000Z', 'lasttime': '2018-10-03T15:00:00.000Z', 'lastupdt': '2018-10-03T15:00:00.000Z', 'netid': 'FA:8F:CA:72:8B:42', 'name': None, 'type': 'infra', 'comment': None, 'wep': '?', 'bcninterval': 0, 'freenet': '?', 'dhcp': '?', 'paynet': '?', 'userfound': False, 'channel': 6, 'encryption': 'unknown', 'country': 'AU', 'region': 'Victoria', 'housenumber': '124', 'road': 'Rathmines Road', 'city': None, 'postalcode': '3123'}, {'trilat': -37.82095718, 'trilong': 145.05506897, 'ssid': None, 'qos': 2, 'transid': '20180605-00000', 'firsttime': '2018-06-05T18:00:00.000Z', 'lasttime': '2018-10-03T15:00:00.000Z', 'lastupdt': '2018-10-03T15:00:00.000Z', 'netid': 'FA:8F:CA:89:88:9C', 'name': None, 'type': 'infra', 'comment': None, 'wep': '?', 'bcninterval': 0, 'freenet': '?',

{'success': True, 'totalResults': 611, 'first': 1, 'last': 11, 'resultCount': 11, 'results': [{'trilat': -37.82051849, 'trilong': 145.05123901, 'ssid': None, 'qos': 0, 'transid': '20181003-00000', 'firsttime': '2018-10-03T18:00:00.000Z', 'lasttime': '2018-10-03T15:00:00.000Z', 'lastupdt': '2018-10-03T15:00:00.000Z', 'netid': 'FA:8F:CA:72:8B:42', 'name': None, 'type': 'infra', 'comment': None, 'wep': '?', 'bcninterval': 0, 'freenet': '?', 'dhcp': '?', 'paynet': '?', 'userfound': False, 'channel': 6, 'encryption': 'unknown', 'country': 'AU', 'region': 'Victoria', 'housenumber': '124', 'road': 'Rathmines Road', 'city': None, 'postalcode': '3123'}, {'trilat': -37.82095718, 'trilong': 145.05506897, 'ssid': None, 'qos': 2, 'transid': '20180605-00000', 'firsttime': '2018-06-05T18:00:00.000Z', 'lasttime': '2018-10-03T15:00:00.000Z', 'lastupdt': '2018-10-03T15:00:00.000Z', 'netid': 'FA:8F:CA:89:88:9C', 'name': None, 'type': 'infra', 'comment': None, 'wep': '?', 'bcninterval': 0, 'freenet': '?',

{'success': True, 'totalResults': 611, 'first': 1, 'last': 11, 'resultCount': 11, 'results': [{'trilat': -37.82051849, 'trilong': 145.05123901, 'ssid': None, 'qos': 0, 'transid': '20181003-00000', 'firsttime': '2018-10-03T18:00:00.000Z', 'lasttime': '2018-10-03T15:00:00.000Z', 'lastupdt': '2018-10-03T15:00:00.000Z', 'netid': 'FA:8F:CA:72:8B:42', 'name': None, 'type': 'infra', 'comment': None, 'wep': '?', 'bcninterval': 0, 'freenet': '?', 'dhcp': '?', 'paynet': '?', 'userfound': False, 'channel': 6, 'encryption': 'unknown', 'country': 'AU', 'region': 'Victoria', 'housenumber': '124', 'road': 'Rathmines Road', 'city': None, 'postalcode': '3123'}, {'trilat': -37.82095718, 'trilong': 145.05506897, 'ssid': None, 'qos': 2, 'transid': '20180605-00000', 'firsttime': '2018-06-05T18:00:00.000Z', 'lasttime': '2018-10-03T15:00:00.000Z', 'lastupdt': '2018-10-03T15:00:00.000Z', 'netid': 'FA:8F:CA:89:88:9C', 'name': None, 'type': 'infra', 'comment': None, 'wep': '?', 'bcninterval': 0, 'freenet': '?',

{'success': True, 'totalResults': 611, 'first': 1, 'last': 11, 'resultCount': 11, 'results': [{'trilat': -37.82051849, 'trilong': 145.05123901, 'ssid': None, 'qos': 0, 'transid': '20181003-00000', 'firsttime': '2018-10-03T18:00:00.000Z', 'lasttime': '2018-10-03T15:00:00.000Z', 'lastupdt': '2018-10-03T15:00:00.000Z', 'netid': 'FA:8F:CA:72:8B:42', 'name': None, 'type': 'infra', 'comment': None, 'wep': '?', 'bcninterval': 0, 'freenet': '?', 'dhcp': '?', 'paynet': '?', 'userfound': False, 'channel': 6, 'encryption': 'unknown', 'country': 'AU', 'region': 'Victoria', 'housenumber': '124', 'road': 'Rathmines Road', 'city': None, 'postalcode': '3123'}, {'trilat': -37.82095718, 'trilong': 145.05506897, 'ssid': None, 'qos': 2, 'transid': '20180605-00000', 'firsttime': '2018-06-05T18:00:00.000Z', 'lasttime': '2018-10-03T15:00:00.000Z', 'lastupdt': '2018-10-03T15:00:00.000Z', 'netid': 'FA:8F:CA:89:88:9C', 'name': None, 'type': 'infra', 'comment': None, 'wep': '?', 'bcninterval': 0, 'freenet': '?',

{'success': False, 'message': 'too many queries today'}
There are no more records
{'success': False, 'message': 'too many queries today'}
There are no more records
{'success': False, 'message': 'too many queries today'}
There are no more records
{'success': False, 'message': 'too many queries today'}
There are no more records
{'success': False, 'message': 'too many queries today'}
There are no more records
{'success': False, 'message': 'too many queries today'}
There are no more records
{'success': False, 'message': 'too many queries today'}
There are no more records
{'success': False, 'message': 'too many queries today'}
There are no more records
{'success': False, 'message': 'too many queries today'}
There are no more records
{'success': False, 'message': 'too many queries today'}
There are no more records
{'success': False, 'message': 'too many queries today'}
There are no more records
{'success': False, 'message': 'too many queries today'}
There are no more records
{'success': Fals

In [31]:
payload

{'onlymine': 'false',
 'freenet': 'false',
 'paynet': 'false',
 'road': 'Rathmines Road',
 'searchAfter': '220252722921052',
 'api_key': 'AID18c60572f3b545f8be5573d53e3e06090ae1905e583e68160d759c2da6f02c4b'}